In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine, text
from sqlalchemy.exc import OperationalError
from dotenv import load_dotenv
import psycopg2
folder_path = 'excel files'


In [2]:
dataframes_dict = {}

for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        file_path = os.path.join(folder_path, filename)    
        df = pd.read_excel(file_path,None) 
        dataframes_dict[filename] = df
        print(dataframes_dict[filename].keys()) 
        


dict_keys(['2022-23', '2023-24'])
dict_keys(['2022-23', '2023-24'])
dict_keys(['2022-23', 'Summary of 2022-23'])


In [3]:
# dataframes_dict['Book_Chapter_Publication.xlsx']['2022-23']
# dataframes_dict['Book_Chapter_Publication.xlsx']['2023-24']
# dataframes_dict['Conference_Publication.xlsx']['2022-23']
# dataframes_dict['Conference_Publication.xlsx']['2023-24']
# dataframes_dict['Journal_Papers.xlsx']['2022-23']
# dataframes_dict['Journal_Papers.xlsx']['Summary of 2022-23']

In [4]:
for key, value in dataframes_dict.items():
    print("File: ",key)
    for k,v in value.items():
        print("\tSheet: ",k)
        

File:  Book_Chapter_Publication.xlsx
	Sheet:  2022-23
	Sheet:  2023-24
File:  Conference_Publication.xlsx
	Sheet:  2022-23
	Sheet:  2023-24
File:  Journal_Papers.xlsx
	Sheet:  2022-23
	Sheet:  Summary of 2022-23


In [5]:
del dataframes_dict['Journal_Papers.xlsx']['Summary of 2022-23']

In [6]:
dataframes_dict['Book_Chapter_Publication.xlsx']['2023-24'].drop('Download File', axis=1,inplace=True)
dataframes_dict['Book_Chapter_Publication.xlsx']['2022-23'].drop('Download File', axis=1,inplace=True)

In [7]:
dataframes_dict['Book_Chapter_Publication.xlsx']['2023-24'].columns

Index(['id', 'Book Title', 'Book Chapter Title', 'Name of Book Publisher',
       'Month', 'Year', 'Year of Pub.', 'ISBNNo No', 'Publication Type',
       'Publication City', 'Author', 'Department Name'],
      dtype='object')

In [8]:
excel_files=list(dataframes_dict.keys())
# Gives list of all the excel files
excel_files

['Book_Chapter_Publication.xlsx',
 'Conference_Publication.xlsx',
 'Journal_Papers.xlsx']

In [9]:
def split_authors(df, column_name, max_authors=10):
    if column_name in df.columns:
        df.loc[:,column_name] = df[column_name].str.strip().str.replace(r',+', ',', regex=True).str.rstrip(',')
    else:
        return df

    # Split the 'author' column into lists of authors
    authors_split = df[column_name].str.split(',', expand=True)
    print(authors_split.shape[0])
    print(authors_split.shape[1])
    # Rename columns to 'author_1', 'author_2', ..., 'author_10'
    authors_split.columns = [f'{column_name}{i+1}' for i in range(authors_split.shape[1])]
    
    # If there are less columns than max_authors, add additional columns with None
    for i in range(authors_split.shape[1], max_authors):
        authors_split[f'{column_name}{i+1}'] = None
    
    # # Trim to max_authors columns
    # authors_split = authors_split.iloc[:, :max_authors] # if more than 10 authors are there than it will remove the extra authors
    
    # Drop the original 'author' column and concatenate the new columns
    df = df.drop(columns=[column_name])
    df = pd.concat([df, authors_split], axis=1)
    
    return df
df = split_authors(dataframes_dict['Book_Chapter_Publication.xlsx']['2022-23'], 'Author', max_authors=10)


42
6


In [10]:
for filename, value in dataframes_dict.items():
    # print("File: ",filename)
    sheets=list(value.keys())
    cols=list(value[sheets[0]].columns)
    # print(cols)
    for k,v in value.items():
        # print(v.columns)
        if list(v.columns)!=cols:
            print("Error: Different columns in sheet",k)

In [28]:
Tables={}
for filename, value in dataframes_dict.items():
    # print("File: ",filename)
    sheets=list(value.keys())
    cols=list(value[sheets[0]].columns)
    # print(cols)
    Dataframe=value[sheets[0]].drop('id',axis=1)
    for i in range(1,len(sheets)):
        Dataframe=pd.concat([Dataframe,value[sheets[i]].drop('id',axis=1)],ignore_index=True).drop_duplicates()
    Tables[filename]=Dataframe
        
        

In [34]:
Tables['Journal_Papers.xlsx']['Unnamed: 24'].unique()

array([nan,
       'Note that from the first issue of 2016, this journal uses article numbers instead of page numbers. See further details here.'],
      dtype=object)

In [133]:
df.head()

,id,Book Title,Book Chapter Title,Name of Book Publisher,Month,Year,Year of Pub.,ISBNNo No,Publication Type,Publication City,...,Author1,Author2,Author3,Author4,Author5,Author6,Author7,Author8,Author9,Author10
0,1,LECTURE NOTES IN NETWORKS AND SYSTEMS,CSBRCA: CLOUD SECURITY BREACHES AND ITS ROOT C...,SPRINGER,7,2022,2022-23,978-981-19-1142-2,Chapter in Book,Singapore,...,CHUDASAMA VIPUL HASMUKHLAL,MADHURI BHAVSAR,VIVEK KUMAR PRASAD,ASHEESH SHAH,AKSHAY MEWADA,None,None,None,None,None
1,2,PROCEEDINGS OF THIRD INTERNATIONAL CONFERENCE ...,A SENTIMENT ANALYSIS-BASED RECOMMENDER FRAMEWO...,"SPRINGER, SINGAPORE",7,2022,2022-23,978-981-19-1141-5,Chapter in Proceeding,Singapore,...,AKHIL BHATIA,SUDEEP TANWAR,ANANSHA ASTHANA,None,None,None,None,None,None,None
2,3,2022 IEEE BOMBAY SECTION SIGNATURE CONFERENCE ...,AI BASED CLASSIFICATION FOR AUTISM SPECTRUM DI...,IEEE,2,2023,2022-23,978-1-6654-9291-1,Chapter in Proceeding,Bombay,...,VERMA JAIPRAKASH VINODKUMAR,SWATI JAIN,SHIVANI PANDYA,None,None,None,None,None,None,None
3,4,HORIZONS IN COMPUTER SCIENCE RESEARCH,THE FUTURE OF INTRUSION DETECTION SYSTEM IN CL...,NOVA SCIENCE PUBLISHERS VOLUME 23,10,2022,2022-23,21592012,Chapter in Book,Newyork,...,SUNIL GAUTAM,None,None,None,None,None,None,None,None,None
4,5,HORIZONS IN COMPUTER SCIENCE RESEARCH,INTRUSION DETECTION SYSTEMS FOR ADVANCED ECOSY...,NOVA SCIENCE PUBLISHERS VOLUME 23,10,2022,2022-23,21592012,Chapter in Book,Newyork,...,SUNIL GAUTAM,None,None,None,None,None,None,None,None,None


In [134]:
load_dotenv()


True

In [135]:
# Function to create a PostgreSQL database and insert data into tables
def create_db_and_insert_data(db_name, sheet_data):
    # Create a connection to PostgreSQL
    engine = create_engine(f'postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}')
    for sheet_name, df in sheet_data.items():
        print(sheet_name)
        df.to_sql(sheet_name, engine, if_exists='replace', index=False)

In [136]:
for i in excel_files:
    for df in dataframes_dict[i]:
        print(i)
        dataframes_dict[i][df]=split_authors(dataframes_dict[i][df], 'Author', max_authors=10)

Book_Chapter_Publication.xlsx
42
6
Book_Chapter_Publication.xlsx
28
5
Conference_Publication.xlsx
75
8
Conference_Publication.xlsx
49
7
Journal_Papers.xlsx


In [ ]:
for i in excel_files:
    create_db_and_insert_data(i[0:-5].lower(),dataframes_dict[i])

In [ ]:
# SELECT 'CREATE DATABASE exp_db' 
# WHERE NOT EXISTS (SELECT FROM pg_database WHERE datname = 'exp_db')